In [ ]:
import os
from libv3.utils import *
import pandas as pd
import json

In [ ]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config('lora_ducy_config')   ### config stored in libv3/exp_config.txt

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

In [ ]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

In [ ]:
paths_traces[0]    ## 14, 17

In [ ]:
## checking if all keys are constant
# for i in range(len(varlist_path)):
varlist1 = read_json(varlist_path[0])
varlist2 = read_json(varlist_path[1])
# varlist3 = read_json(varlist_path[2])
# varlist4 = read_json(varlist_path[3])
# print(varlist.keys())
keys = list(varlist1.keys())
keys.sort()
print(keys)
# print(len(varlist1.keys()), len(varlist2.keys()), len(varlist3.keys()), len(varlist4.keys()))
print(len(varlist1.keys()), len(varlist2.keys()))

# for k in keys:
#     print(varlist1[k], varlist2[k], varlist3[k], varlist4[k])
#     # print(k)
#     if not (varlist1[k] == varlist2[k] == varlist3[k] == varlist4[k]): 
#         print(k)

##########################################################

for vl in varlist_path:
    varlist = read_json(vl)
    print(len(varlist.keys()))

    # for k in varlist.keys():
    #     print(k, varlist[k])


In [ ]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    check_con, _ = is_consistent(varlist_path)

    if check_con != False:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        print('varlist is not consistent')


In [ ]:
to_number

In [ ]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

In [ ]:
sorted_keys

## Trace Plots

### Process Traces

In [ ]:
########## process the traces ###########
col_data = preprocess_traces(paths_traces, var_list)   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

### Generate plot trace data

In [ ]:
############### preprocessing data to plot using plotly ##############
'''
Restructure the data in dictionary with (keys,value) pair :-  (time, timestamps) , (trace_name, trace)
'''
all_df = get_dataframe(col_data)   

In [ ]:
all_df[2]

In [ ]:

### plot single trace
for i, df in enumerate(all_df):
    
    if i == trace_file:
        plot_single_trace(df, sorted_keys, with_time=False, is_xticks=False)

In [ ]:
### get timestamp

timestamp = index2timestamp(all_df[0], 227)
print(timestamp)

## Execution Interval Plots

### Preprocess data

In [ ]:
####### preprocessing data ########

#### extract timestamps for each variable and store them in a dictionary along with index values for each variable in event trace

var_timestamps = get_var_timestamps(paths_traces)    #### in format (filename, dict of timestamps and index values)

to_plot = preprocess_variable_plotting(var_timestamps, var_list, from_number, trace_number=trace_file)   ### restructure the data for plotting


In [ ]:
len(to_plot[6][-1])
to_plot[6][-1]

### Generate execution interval plots

In [ ]:
### plot the data
plot_execution_interval_single(to_plot, is_xticks=False)

## Plot Labels

In [ ]:
### count and prepare labels to plot
'''
labels are of format [index1, index2, timestamp1, timestamp2, class]
'''
class_count = defaultdict(int)
for i, path in enumerate(paths_label):
    label_content = prepare_gt(path)
    ind, ts, cls = label_content
    # print(ind, ts, cls)
    for c in cls:
        class_count[c]+=1
        
    if i == trace_file:
        print(path)
        toplot_gt = label_content

    print(os.path.split(path)[-1], class_count)

    # break
for key, val in class_count.items():
    print(key, val)

In [ ]:
### plot
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_obj = plot_single_trace(df, var_list, with_time=False, is_xticks=True, ground_truths=toplot_gt)
        plot_obj.show()

## Generate train data

In [ ]:
# '''
# Add the indices for the interval in 'normal_seq_inter' that show correct behvaiour in the traces. 
# The format is as follows:
# path_traces = [path1, path2, ...]
# normal_seq_inter = [ ( intervals for trace1, ...), 
#                     ( intervals for trace2 ...),
#                       ...] 
# '''

# normal_seq_inter = (((0,6200), (6500,9910)),
#                     ((0,8260), (8260,11000)))  ### v3, mamba2 trial1, trial2
            

# for p, n_inter in zip(paths_traces, normal_seq_inter):
#     trace = read_traces(p)
#     train_data_path = os.path.join(os.path.dirname(p), 'train_data')

#     if not os.path.exists(train_data_path):
#         os.makedirs(train_data_path)

#     for i, inter in enumerate(n_inter):
#         start, end = inter
#         # trace[start:end].to_csv(os.path.join(train_data_path, f'interval_{start}_{end}.csv'), index=False)
#         json.dump(trace[start:end], open(os.path.join(train_data_path, f'interval_{start}_{end}.json'), 'w'))

In [ ]:
paths_traces

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import matplotlib.dates as mdates
import datetime
import pandas as pd

# Parameters
days = 7
hours_per_day = 24
total_hours = days * hours_per_day  # Total number of data points for 7 days
sensors = 4

# Generate base temperature data for each sensor (normal values between 20°C and 30°C)
np.random.seed(42)  # For reproducibility
base_temperature = np.random.uniform(20, 30, size=(total_hours, sensors))

# Introduce random spikes (anomalies) in the data with a value of 100
num_anomalies = 10  # Number of anomalies to introduce
for _ in range(num_anomalies):
    sensor = random.randint(0, sensors - 1)  # Randomly choose a sensor
    time_point = random.randint(0, total_hours - 1)  # Randomly choose a time point
    base_temperature[time_point, sensor] = 100  # Set the anomaly value to 100

# Generate time points (hours) for 7 days as datetime objects for proper formatting
start_time = datetime.datetime(2022, 7, 22, 4, 48)  # Set a starting datetime
time_points = [start_time + datetime.timedelta(hours=i) for i in range(total_hours)]

# Save the data to an Excel file
df = pd.DataFrame(base_temperature, columns=[f'Sensor {i+1}' for i in range(sensors)])
df['Time'] = time_points
df = df[['Time'] + [f'Sensor {i+1}' for i in range(sensors)]]  # Reorder to put 'Time' first
df.to_excel('sensor_temperature_data_7days.xlsx', index=False)  # Save the DataFrame to Excel

# Set up the figure and axis for plotting
fig, ax = plt.subplots(figsize=(12, 6))

# Plot temperature data for each sensor
for sensor in range(sensors):
    ax.plot(time_points, base_temperature[:, sensor], label=f'Sensor {sensor + 1}', linewidth=2)

# Customize time format on x-axis
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))  # Show a tick per day
ax.xaxis.set_minor_locator(mdates.HourLocator(interval=12))  # Minor ticks every 12 hours
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y\n%H:%M'))  # Format as date and time

# Rotate date labels for better readability
plt.xticks()

# Labeling the axes
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Temperature (°C)', fontsize=12)

# Set Y-axis limit
ax.set_ylim([0, 100])

# Adding gridlines and a legend
ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=sensors)

# Set title and show the plot
# plt.title('Temperature Readings from 4 Sensors Over 7 Days with Anomalies', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Parameters
days = 7
hours_per_day = 24
total_hours = days * hours_per_day  # Total number of data points for 7 days
sensors = 4

# Generate base temperature data for each sensor (normal values between 20°C and 30°C)
np.random.seed(42)  # For reproducibility
base_temperature = np.random.uniform(20, 30, size=(total_hours, sensors))

# Introduce random spikes (anomalies) in the data with a value of 100
num_anomalies = 10  # Number of anomalies to introduce
for _ in range(num_anomalies):
    sensor = random.randint(0, sensors - 1)  # Randomly choose a sensor
    time_point = random.randint(0, total_hours - 1)  # Randomly choose a time point
    base_temperature[time_point, sensor] = 100  # Set the anomaly value to 100

# Generate time points (hours) for 7 days
time_points = np.arange(total_hours)

# Set up the figure and axis for plotting
plt.figure(figsize=(12, 6))

# Plot temperature data for each sensor
for sensor in range(sensors):
    plt.plot(time_points, base_temperature[:, sensor], label=f'Sensor {sensor + 1}')

# Labeling the axes
plt.xlabel('Time (hours)', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)

# Adding gridlines and a legend
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()

# Set title and show the plot
plt.title('Temperature Readings from 4 Sensors Over 7 Days with Anomalies at 100°C', fontsize=14)
plt.tight_layout()
plt.show()
